In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import argparse
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import sys
sys.path.append('..')
from models.common_autoencoder_blocks import Encoder  
from models.cifar10_supconv import CIFAR10SupCon
from trainers.supcon_trainer import SupConTrainer
from trainers.classifier_trainer import ClassifierTrainer
from models.cifar10_classifier import CIFAR10Classifier
import torch.backends.cudnn as cudnn
import random
import numpy as np

seed = 42 

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
cudnn.deterministic = True
cudnn.benchmark = False

device = 'cuda' if torch.cuda.is_available() else 'cpu'
class TwoCropsTransform:
    def __init__(self, base_transform):
        self.base_transform = base_transform

    def __call__(self, x):
        im1 = self.base_transform(x)
        im2 = self.base_transform(x)
        return im1, im2

data_path = "/datasets/cv_datasets/data"

base_transform = transforms.Compose([
    transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), shear=5),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.0))], p=0.2),
    transforms.ToTensor(),
])

train_transform = TwoCropsTransform(base_transform)
val_base_transform = transforms.Compose([transforms.ToTensor()])
val_transform = TwoCropsTransform(val_base_transform)

train_dataset = datasets.CIFAR10(root=data_path, train=True, download=True, transform=train_transform)
val_dataset = datasets.CIFAR10(root=data_path, train=False, download=True, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True, num_workers=2, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=2048, shuffle=False, num_workers=2, drop_last=True)

if torch.cuda.is_available():
    print('Using GPU')
else:
    raise Exception('GPU not found')

supcon = CIFAR10SupCon(
    input_shape=(3, 32, 32),
    channels=[64, 128,256,512],
    kernel_sizes=[3, 3, 3,3],
    strides=[2, 2, 2,2],
    paddings=[1, 1, 1,1],
    latent_dim=128,
    hidden_dims=[1024],
    batch_norm_conv=True
)
optimizer = torch.optim.Adam(supcon.parameters(), lr=1e-4, weight_decay=1e-4)

supcon_trainer = SupConTrainer(
    model=supcon,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    device=device,
    num_epochs=200,
    patience=10,
    temperature=0.07,
    scheduler_type=

)
supcon_trainer.train()
supcon_trainer.plot_metrics()



Using GPU
[Epoch 1] TrainLoss=8.2270, ValLoss=8.2543
[Epoch 2] TrainLoss=8.0940, ValLoss=8.0263
[Epoch 3] TrainLoss=8.0134, ValLoss=7.9240
[Epoch 4] TrainLoss=7.9606, ValLoss=7.8789
[Epoch 5] TrainLoss=7.9236, ValLoss=7.8398
[Epoch 6] TrainLoss=7.8852, ValLoss=7.8321
[Epoch 7] TrainLoss=7.8511, ValLoss=7.7926
[Epoch 8] TrainLoss=7.8224, ValLoss=7.7607
[Epoch 9] TrainLoss=7.7927, ValLoss=7.7351
[Epoch 10] TrainLoss=7.7647, ValLoss=7.7102
[Epoch 11] TrainLoss=7.7360, ValLoss=7.6811
[Epoch 12] TrainLoss=7.7123, ValLoss=7.6437
[Epoch 13] TrainLoss=7.6901, ValLoss=7.6414
[Epoch 14] TrainLoss=7.6602, ValLoss=7.6284
[Epoch 15] TrainLoss=7.6373, ValLoss=7.5942
[Epoch 16] TrainLoss=7.6129, ValLoss=7.5929
[Epoch 17] TrainLoss=7.5939, ValLoss=7.5559
[Epoch 18] TrainLoss=7.5706, ValLoss=7.5479
[Epoch 19] TrainLoss=7.5486, ValLoss=7.5537
[Epoch 20] TrainLoss=7.5273, ValLoss=7.5229
[Epoch 21] TrainLoss=7.5128, ValLoss=7.5016


In [ ]:
classifier_train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), shear=5),
    transforms.ToTensor(),
])

classifier_val_transform = val_base_transform  


train_dataset_calssification = datasets.CIFAR10(root=data_path, train=True, download=True, transform=classifier_train_transform)
val_dataset_calssification   = datasets.CIFAR10(root=data_path, train=False, download=True, transform=classifier_val_transform)
train_calssification_loader = DataLoader(train_dataset_calssification, batch_size=128, shuffle=True)
val_calssification_loader = DataLoader(val_dataset_calssification, batch_size=128, shuffle=False)
classifier = CIFAR10Classifier(latent_dim=128, num_classes=10, dropout_fc = 0.2, batch_norm_fc=True) 
classifier_trainer = ClassifierTrainer(
    encoder=supcon,
    classifier=classifier,
    train_loader=train_calssification_loader,
    val_loader=val_calssification_loader,
    device=device,
    lr=5e-5,
    weight_decay=3e-4,
    num_epochs=60,
    patience=10,
)
classifier_trainer.train()
classifier_trainer.plot_metrics()

Epoch [1] - TrainAcc: 0.3197, ValAcc: 0.5666 | ValLoss: 1.3584
Epoch [2] - TrainAcc: 0.3832, ValAcc: 0.5846 | ValLoss: 1.2873
Epoch [3] - TrainAcc: 0.3941, ValAcc: 0.5862 | ValLoss: 1.2624
Epoch [4] - TrainAcc: 0.3997, ValAcc: 0.5888 | ValLoss: 1.2362
Epoch [5] - TrainAcc: 0.4084, ValAcc: 0.5897 | ValLoss: 1.2293
Epoch [6] - TrainAcc: 0.4133, ValAcc: 0.5934 | ValLoss: 1.2247
Epoch [7] - TrainAcc: 0.4202, ValAcc: 0.5889 | ValLoss: 1.2240
Epoch [8] - TrainAcc: 0.4214, ValAcc: 0.5897 | ValLoss: 1.2228
Epoch [9] - TrainAcc: 0.4245, ValAcc: 0.5903 | ValLoss: 1.2097
Epoch [10] - TrainAcc: 0.4273, ValAcc: 0.5916 | ValLoss: 1.2111
Epoch [11] - TrainAcc: 0.4290, ValAcc: 0.5892 | ValLoss: 1.2039
Epoch [12] - TrainAcc: 0.4291, ValAcc: 0.5898 | ValLoss: 1.2088
Epoch [13] - TrainAcc: 0.4378, ValAcc: 0.5894 | ValLoss: 1.2028
Epoch [14] - TrainAcc: 0.4320, ValAcc: 0.5905 | ValLoss: 1.2081
Epoch [15] - TrainAcc: 0.4370, ValAcc: 0.5946 | ValLoss: 1.1946
Epoch [16] - TrainAcc: 0.4357, ValAcc: 0.5954 | V

In [ ]:
from utils import plot_tsne_supcon
plot_tsne_supcon(supcon, val_calssification_loader, device='cuda')